In [27]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import csv
import re
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import string
from umap import UMAP
#import umap.umap_ as UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Carico il modello sentence transformer

# Carica il modello BERT italiano di nickprock
model_name = "nickprock/sentence-bert-base-italian-uncased"
embedding_model  = SentenceTransformer(model_name)

# salva il modello
modelPath = r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\nickprock_sent_trans"
embedding_model.save(modelPath)

In [2]:
# carica il modello
modelPath = r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\nickprock_sent_trans"
embedding_model = SentenceTransformer(modelPath)

c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Carico i commenti

In [3]:
path = "E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo\Facebook\csv_docs_def\comments4_no_spam_nan.csv"
comments_no_nan = pd.read_csv(path, parse_dates=["post_date"], quotechar='"', quoting=csv.QUOTE_ALL, encoding="utf-8", low_memory=False)

In [4]:
def replace_dots(text):
    # This pattern will match sequences of words separated by dots but not acronyms.
    # Acronyms are sequences of single uppercase letters separated by dots and potentially followed by a dot.
    #pattern = r'\b(?:\w+\.\w{2,}|\w{2,}\.\w+)(?:\.\w+)*\b'
    pattern = r'\b(?:[a-zA-Z]+\.[a-zA-Z]{2,}|[a-zA-Z]{2,}\.[a-zA-Z]+)(?:\.[a-zA-Z]+)*\b'
    #pattern = r'\b(?!\d{1,2}\.\d{1,2}\.\d{2,4}\b)(?=\w*[^\d]\w*)(?:\w{2,}\.\w+|\w+\.\w{2,})(?:\.\w+)*\b'

    def replacement(match):
        # Replace dots with spaces in the matched string
        return match.group(0).replace('.', '. ')
    
    # Use re.sub to replace the matched pattern with the replacement function
    return re.sub(pattern, replacement, text)

def replace_commas(text):
    # This pattern will match sequences of words separated by dots but not acronyms.
    # Acronyms are sequences of single uppercase letters separated by dots and potentially followed by a dot.
    #pattern = r'\b(?:\w+\.\w{2,}|\w{2,}\.\w+)(?:\.\w+)*\b'
    pattern = r'\b(?:[a-zA-Z]+,[a-zA-Z]{2,}|[a-zA-Z]{2,},[a-zA-Z]+)(?:,[a-zA-Z]+)*\b'
    #pattern = r'\b(?!\d{1,2}\.\d{1,2}\.\d{2,4}\b)(?=\w*[^\d]\w*)(?:\w{2,}\.\w+|\w+\.\w{2,})(?:\.\w+)*\b'

    def replacement(match):
        # Replace dots with spaces in the matched string
        return match.group(0).replace(',', ', ')
    
    # Use re.sub to replace the matched pattern with the replacement function
    return re.sub(pattern, replacement, text)

def clean_text(text):
    # Rimuove gli articoli con apostrofi come "l'", "all'", ecc.
    cleaned_text = re.sub(r"^(l|all|d|nell|sull|coll|dall|dell|all|quell|quest)'", '', text)
    return cleaned_text

def remove_leading_point(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?:(?<=\s)|(?<=^))\.\b(?=\w+)'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, '', text)

    return result

def remove_ending_point(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?:\b)\.$'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, '', text)

    return result

def remove_hyphen(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?<=[a-zA-Z])-(?=[a-zA-Z])'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, ' ', text)

    return result

def remove_link(text):   
    
    pattern = r'\bhttps?\S+|\bcom/share\S+|\bstory_fbid=\S+|\bcom/story\S+'
    result = re.sub(pattern, '', text)

    return result

def transform_text(text):
    text = str(text).lower()
    #text = unidecode(text)
    text = replace_dots(text)
    text = replace_commas(text)
    text = remove_leading_point(text)
    text = remove_hyphen(text)
    text = remove_link(text)
    return text

In [5]:
transform_text("Fresco? C'è il rischio che non rimanga nessun frutto tra grandine, neve e gelo,speriamo bene.")

"fresco? c'è il rischio che non rimanga nessun frutto tra grandine, neve e gelo, speriamo bene."

In [6]:
def remove_receiver(row):
    if row["level"] > 0 and isinstance(row["content"], str):
        receiver_name_len = len(row["type"].split()) - len(row["author"].split()) - 7
        if len(row["content"].split()) >= receiver_name_len:
            token_list = row["content"].split()
            del token_list[:receiver_name_len]
            content_without_receiver = " ".join(token_list)
            row["content"] = transform_text(content_without_receiver)
    return row

# Apply the function to each row
comments_truncated = comments_no_nan.apply(remove_receiver, axis=1)

In [7]:
# carico la collezione di documenti e filtro per lunghezza
#from nltk.corpus import stopwords

#tokenized_text = []
#tokenized_filtered_text = []
#stopwords = set(stopwords.words('italian'))
#stopwords.update(string.punctuation)
comments_with_index = []
comments = []
for index, row in comments_truncated.iterrows():
    #tokenized_comment = nltk.word_tokenize(str(row["content"]))
    #filtered_tokens = [word for word in tokenized_comment if word.lower() not in stopwords]
    #tokenized_filtered_text.append((index, filtered_tokens))
    #tokenized_text.append((index, tokenized_comment))
    if len(row["content"].split()) >= 20: 
        comments_with_index.append((index, transform_text(str(row["content"]))))
        comments.append(transform_text(str(row["content"])))


In [8]:
len(comments)

27156

In [31]:
comments

['questo io non sono in grado di stabilirlo, posso dire che mio suocero mi raccontava che il mese di giugno negli anni 40/ta in molise gelò il grano.',
 "da un po' di tempo non si parla più di surriscaldamento ma di cambiamento climatici. ottimo stratagemma così sia che faccia più freddo o che faccia più caldo o che piova meno o che piova di più, la responsabilità è sempre del nostro comportamento. mio cognato si è sposato nell'aprile del 1985. ricordo che in quei giorni sul catria è nevicato. io mi sono spostato il 20 gennaio ed et stata una giornata di sole. non calda ma soleggiata. questo per dire che la variazioni ci sono.",
 'e vero io mi sono sposata il 29 dicembre del 74 e uscita una giornata piena di sole faceva caldo',
 'certo , ma non si scioglievano i ghiacciai ....nella mia lunga vita ho visto nevicare a maggio , ho visto gelare il grano , ecc. fenomeni anomali ci sono sempre stati ma ciò che fa la differenza sono le temperature medie.',
 "vero. ma da cosa dipende? nel medi

for comment in comments:
    if len(comment[1].split()) >= 20:
        print(comment[1])

# pre calcolo gli embedding
embeddings = embedding_model.encode(comments, show_progress_bar=True)

In [16]:
#np.save(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\embeddings.npy", embeddings)
embeddings = np.load(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\embeddings.npy")

In [10]:
# istanzio umap così da evitare comportamento stocastico
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [26]:
# istanzio il modello di clustering così da poter controllare (indirettamente con il paramentro min_cluster_size)
# il numero di cluster
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [22]:
# Using the CountVectorizer, we can do a number of things: Remove stopwords, Ignore infrequent words, Increase the n-gram range
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('italian'))
STOPWORDS.update(string.punctuation)
STOPWORDS = list(STOPWORDS)
vectorizer_model = CountVectorizer(stop_words=STOPWORDS, min_df=2, ngram_range=(1, 2))

In [28]:
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

'''# GPT-3.5
client = openai.OpenAI(api_key="sk-...")
prompt = """
I have a topic that contains the following documents: 
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)'''

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}

In [29]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(comments, embeddings)

# Show topics
topic_model.get_topic_info()

2024-06-26 15:30:56,890 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


2024-06-26 15:31:36,928 - BERTopic - Dimensionality - Completed ✓
2024-06-26 15:31:36,928 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-26 15:31:41,243 - BERTopic - Cluster - Completed ✓
2024-06-26 15:31:41,243 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-26 15:33:51,940 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,145,-1_gregge_vertici_democratico_popolo,"[gregge, vertici, democratico, popolo, redditi...","[popolo popolo, popolo devono, popolo deve, po...","[gregge, vertici, democratico, popolo, redditi...","[sardegna, umiliazione, irpef, consegnare, sce...",[in un paese democratico con dei vertici votat...
1,0,26321,0_agricoltori_solo_italia_fare,"[agricoltori, solo, italia, fare, italiani, se...","[gente, vogliono, tanto, dobbiamo, popolo, cos...","[agricoltori, solo, italia, fare, italiani, se...","[fare, senza, essere, quando, terra, anni, fat...",[speriamo che non sia paglia di fuoco. non bas...
2,1,274,1_amicizia_richiesta_ciao_messaggio,"[amicizia, richiesta, ciao, messaggio, profilo...","[richiesta amicizia, ciao, messaggio, molto, g...","[amicizia, richiesta, ciao, messaggio, profilo...","[messaggio, profilo, messenger, spero, grazie,...","[ciao, spero che tu stia bene. vorrei scusarmi..."
3,2,57,2_000_prestito_000 000_prestiti,"[000, prestito, 000 000, prestiti, 000 euro, t...","[000, 000 000, prestiti, 10 000, 500 000, 000 ...","[000, prestito, 000 000, prestiti, 000 euro, t...","[maggiori, message, annuo, ore, , , , , , ]",[siamo una società di credito registrata e ope...
4,3,49,3_the_and_to_of,"[the, and, to, of, is, this, you, can, of the,...","[the farmers, for, this, to the, klaus, klaus ...","[the, and, to, of, is, this, you, can, of the,...","[pages, world, farmers, free, part, european, ...","[von prince/free city of danzig, the author of..."
5,4,40,4_farine insetti_farine_latte formaggio_voglio...,"[farine insetti, farine, latte formaggio, vogl...","[agricoltori carne, cibo agricoltori, mangio p...","[farine insetti, farine, latte formaggio, vogl...","[voglio, , , , , , , , , ]",[sempre al fianco degli agricoltori carne latt...
6,5,38,5_meravigliosamente_grandiiii_perseveranza_int...,"[meravigliosamente, grandiiii, perseveranza, i...","[grandiiii, perseveranza, forti, meravigliosam...","[meravigliosamente, grandiiii, perseveranza, i...","[meravigliosamente, grandiiii, intera, nazione...",[siete l' orgoglio dell' intera nazione !!!! ...
7,6,33,6_les_de_en_et,"[les, de, en, et, interessati, est, disponibil...","[comme, une, voir, faire, les, votre, belgique...","[les, de, en, et, interessati, est, disponibil...","[les, et, est, disponibile, que, consulenza, d...","[actuellement en france et en allemagne, de gr..."
8,7,29,7_donazione_vorrei_donare_persona,"[donazione, vorrei, donare, persona, somma, ma...","[qualcuno germania, vorrei sapere, volete davv...","[donazione, vorrei, donare, persona, somma, ma...","[donazione, vorrei, donare, persona, malattia,...",[beatrice allegretti mi dispiace contattarti i...
9,8,29,8_cura_ore 11_11 00_causato,"[cura, ore 11, 11 00, causato, passate, arriva...","[torniamo, combattere, 11, 11 00, ore 11, insi...","[cura, ore 11, 11 00, causato, passate, arriva...","[cura, passate, parola, combattere, , , , , , ]","[alle ore 11.00 collegatevi sulla pagina, pass..."


In [30]:
topic_model.representative_docs_

{-1: ["in un paese democratico con dei vertici votati dal popolo!se il popolo non è d'accordo su la scelta dei vertici, i vertici essendo a disposizione del popolo devono ascoltarlo!se succedono queste cose, che il popolo deve lottare per essere ascoltato, non siamo in democrazia!queste scene sono assurde in un mondo democratico !!",
  "in un paese democratico con dei vertici votati dal popolo!se il popolo non è d'accordo su la scelta dei vertici, i vertici essendo a disposizione del popolo devono ascoltarlo!se succedono queste cose, che il popolo deve lottare per essere ascoltato, non siamo in democrazia!queste scene sono assurde in un mondo democratico !!",
  "in un paese democratico con dei vertici votati dal popolo!se il popolo non è d'accordo su la scelta dei vertici, i vertici essendo a disposizione del popolo devono ascoltarlo!se succedono queste cose, che il popolo deve lottare per essere ascoltato, non siamo in democrazia!queste scene sono assurde in un mondo democratico !!"],